# A2


In [2]:
# Author: 黄宇康
# Time: 2023/02/27

import datetime

print(datetime.datetime.today())

2023-02-27 21:01:11.301360


In [3]:
import numpy as np
import scipy
import math

In [4]:
# 参数初始化
SIGMA = 50.0  # 惩罚因子
EPSILON = 1e-3  # 终止条件
TOL = 1e-3  # tolerance
MAX_N = 1000  # 最大迭代次数

# 增广拉格朗日方法中的参数
ALPHA = 0.4
BETA = 0.7
ROU = 1.5

e = 1e-10

In [5]:
# A2
# 目标函数
def objfunc(x):      
    return x[0] * x[3] * ( x[0] + x[1] + x[2] ) + x[2]

# 等式约束
def eq_con_func1(x):
    return x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2 - 40

# 不等式约束
def ieq_con_func1(x):
    return - (x[0] * x[1] * x[2] * x[3]) + 25 + e

def ieq_con_func2(x):
    return x[0] - 5 + e

def ieq_con_func3(x):
    return x[1] - 5 + e

def ieq_con_func4(x):
    return x[2] - 5 + e

def ieq_con_func5(x):
    return x[3] - 5 + e

def ieq_con_func6(x):
    return -x[0] + 1 + e

def ieq_con_func7(x):
    return -x[1] + 1 + e

def ieq_con_func8(x):
    return -x[2] + 1 + e

def ieq_con_func9(x):
    return -x[3] + 1 + e


eq_con_func = [eq_con_func1]
ieq_con_func = [ieq_con_func1, ieq_con_func2, ieq_con_func3, ieq_con_func4, ieq_con_func5, ieq_con_func6, ieq_con_func7, ieq_con_func8, ieq_con_func9]

In [6]:
# calculate norm
def cal_vec_norm(x, order=2):
    if(type(x) == np.ndarray):
        if(order == np.Inf):
            return np.max( np.abs(x) )
        else:
            return np.sum((np.abs(x) )**2, axis=0)**(1/order)
    else:
        print("The input is not a ndarray")

In [7]:
# 约束违反度
def vk(x, eq_con_func, ieq_con_func, miu, sigma):

    eq_con_sum = np.sum( [ (eq_con_func(x)**2) for eq_con_func in eq_con_func] )

    ieq_con_sum = np.sum( [(np.max([ieq_con_func(x), -miu/sigma]))**2 for miu, ieq_con_func in zip(miu, ieq_con_func)] )

    return math.sqrt(eq_con_sum + ieq_con_sum)


In [8]:
# Augmented-Lagrange function entrnce
def ALMFunc(x, objfunc, eq_con_func, ieq_con_func, labda, miu, sigma):
    """
    The Augmented-Lagrange function

    Parameters
    ----------
    x:-.
    objfunc: the object func.
    eqfunc: constraint equation.
    ieqfunc: constraint iequation.
    labda: λ, Lagrange multiplex of eqfunc
    miu: μ, Lagrange multiplex of ieqfunc
    sigma: σ, penalty factor
    """

    # contraint equation
    eq_con_sum = np.sum( [labda * eq_con_func(x) + 0.5 * sigma * (eq_con_func(x)**2) for labda, eq_con_func in zip(labda, eq_con_func)] )

    # contraint iequation
    ieq_con_sum = (0.5 * sigma) * ( np.sum( [(np.max([ieq_con_func(x) + miu/sigma, 0]))**2 - (miu/sigma)**2 for miu, ieq_con_func in zip(miu, ieq_con_func)] ) ) 

    # print(objfunc(x), eq_con_sum, ieq_con_sum)

    return objfunc(x) + eq_con_sum + ieq_con_sum

## 调用前面构建的拉格朗日方法完成求解

In [9]:
# step 1
# initialize random labda, miu
labda = np.random.random(1)
miu = np.random.random(9)

n = 0  # 迭代次数
t = 1000
tol_k = 1e-1   # 精度常数
epsilon_k = 100  # 约束违反常数
sigma_k = SIGMA  # 惩罚因子
labda_k = labda   # 等式约束的拉格朗日乘子
miu_k = miu  # 不等式约束的拉格朗日乘子
x_k = np.array([1, 5, 5, 1])  # xk初始值

while( n < 1000 ):

    n = n + 1

    # step 2
    # construct the Augmented-Lagrange function
    ALM_Func = lambda x: ALMFunc(x, objfunc, eq_con_func, ieq_con_func, labda_k, miu_k, sigma_k)

    # step 3
    # 无约束优化问题的求解
    # func: 函数, x0：初值， BFGS：方法
    result = scipy.optimize.minimize(ALM_Func, x_k, tol = tol_k, method='L-BFGS-B', options={'disp': False})

    

    x_k = result.x

    gnorm_k = cal_vec_norm(np.array([objfunc(x_k)]))
    vk_k = vk(x_k, eq_con_func, ieq_con_func, miu_k, sigma_k)

    print("======", n, vk_k, epsilon_k)

    # step 4
    # 更新参数
    if(vk_k <= epsilon_k):
        if(vk_k <= EPSILON):
            break

        labda_k = labda_k + sigma_k * eq_con_func1(x_k)

        for i in range(0, len(miu_k)):
            miu_k[i] = np.max([miu_k[i] + sigma_k*ieq_con_func[i](x_k) , 0])

        sigma_k = sigma_k
        tol_k = tol_k / sigma_k
        epsilon_k = epsilon_k / (sigma_k**BETA)

    else:
        labda_k = labda_k
        sigma_k = ROU * sigma_k
        tol_k = 1.0 / sigma_k
        epsilon_k = epsilon_k / (sigma_k**ALPHA)       


====== 1 0.03803224216836751 100
====== 2 0.01551742034061906 6.467270065773575
====== 3 0.015197433735375257 0.41825582103650943
====== 4 0.02076752663643662 0.02704973351224967
====== 5 0.0009616997377556709 0.0017493793183092458


In [10]:
sum(result.x)
result.x

print("迭代输出xk: ", result.x, "等式约束xk的结果为: ", eq_con_func1(result.x))
print("fx最终值: ", objfunc(result.x))

迭代输出xk:  [0.99904081 4.74444324 3.81899045 1.38109317] 等式约束xk的结果为:  -6.941449030506419e-05
fx最终值:  17.012990960913058


## 使用Scipy测试结果

In [11]:
# scipy调库计算结果

from scipy.optimize import minimize
e = 1e-10
fun = lambda x: x[0] * x[3] * ( x[0] + x[1] + x[2] ) + x[2]  # 目标函数
cons = ({'type':'eq','fun':lambda x:x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2 - 40}, #约束条件
        {'type':'ineq','fun':lambda x:x[0] * x[1] * x[2] * x[3] -25 - e},             #都分别大于0
        {'type':'ineq','fun':lambda x:x[0] - 1 - e},
        {'type':'ineq','fun':lambda x:x[1] - 1 - e},
        {'type':'ineq','fun':lambda x:x[2] - 1 - e},
        {'type':'ineq','fun':lambda x:x[3] - 1 - e},
        {'type':'ineq','fun':lambda x:-x[0] + 5 - e}, 
        {'type':'ineq','fun':lambda x:-x[1] + 5 - e}, 
        {'type':'ineq','fun':lambda x:-x[2] + 5 - e}, 
        {'type':'ineq','fun':lambda x:-x[3] + 5 - e},         
        )

x0 = np.array((1, 5, 5, 1)) #设置xyz的初始值
res = minimize(fun,x0,method='SLSQP',constraints=cons)
print("最小值:",res.fun)
print("最优解:",res.x)
print("迭代终止是否成功",res.success)
print("迭代终止原因",res.message)

最小值: 17.01401724576546
最优解: [1.         4.74299606 3.82115467 1.37940764]
迭代终止是否成功 True
迭代终止原因 Optimization terminated successfully
